<h1><div align="center">Managing Accelerated Application Memory with CUDA C/C++ Unified Memory</div></h1>

![CUDA](https://d29g4g2dyqv443.cloudfront.net/sites/default/files/pictures/2019/NVIDIACuda_Logo.jpg)

The [*CUDA Best Practices Guide*](http://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#memory-optimizations), a highly recommended followup to this and other CUDA fundamentals labs, recommends a design cycle called **APOD**: **A**ssess, **P**arallelize, **O**ptimize, **D**eploy. In short, APOD prescribes an iterative design process, where developers can apply incremental improvements to their accelerated application's performance, and ship their code. As developers become more competent CUDA programmers, more advanced optimization techniques can be applied to their accelerated code bases.

This lab will support such a style of iterative development. You will be using the Nsight Systems command line tool **nsys** to qualitatively measure your application's performance, and to identify opportunities for optimization, after which you will apply incremental improvements before learning new techniques and repeating the cycle. As a point of focus, many of the techniques you will be learning and applying in this lab will deal with the specifics of how CUDA's **Unified Memory** works. Understanding Unified Memory behavior is a fundamental skill for CUDA developers, and serves as a prerequisite to many more advanced memory management techniques.

---
## Prerequisites

To get the most out of this lab you should already be able to:

- Write, compile, and run C/C++ programs that both call CPU functions and launch GPU kernels.
- Control parallel thread hierarchy using execution configuration.
- Refactor serial loops to execute their iterations in parallel on a GPU.
- Allocate and free Unified Memory.

---
## Objectives

By the time you complete this lab, you will be able to:

- Use the Nsight Systems command line tool (**nsys**) to profile accelerated application performance.
- Leverage an understanding of **Streaming Multiprocessors** to optimize execution configurations.
- Understand the behavior of **Unified Memory** with regard to page faulting and data migrations.
- Use **asynchronous memory prefetching** to reduce page faults and data migrations for increased performance.
- Employ an iterative development cycle to rapidly accelerate and deploy applications.

In [ ]:
!nvidia-smi

Sun Jun  8 20:22:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---
## Iterative Optimizations with the NVIDIA Command Line Profiler

The only way to be assured that attempts at optimizing accelerated code bases are actually successful is to profile the application for quantitative information about the application's performance. `nsys` is the Nsight Systems command line tool. It ships with the CUDA toolkit, and is a powerful tool for profiling accelerated applications.

`nsys` is easy to use. Its most basic usage is to simply pass it the path to an executable compiled with `nvcc`. `nsys` will proceed to execute the application, after which it will print a summary output of the application's GPU activities, CUDA API calls, as well as information about **Unified Memory** activity, a topic which will be covered extensively later in this lab.

When accelerating applications, or optimizing already-accelerated applications, take a scientific and iterative approach. Profile your application after making changes, take note, and record the implications of any refactoring on performance. Make these observations early and often: frequently, enough performance boost can be gained with little effort such that you can ship your accelerated application. Additionally, frequent profiling will teach you how specific changes to your CUDA code bases impact its actual performance: knowledge that is hard to acquire when only profiling after many kinds of changes in your code bases.

### Exercise: Profile an Application with nsys

[01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) (<------ you can click on this and any of the source file links in this lab to open them for editing) is a naively accelerated vector addition program. Use the two code execution cells below (`CTRL` + `ENTER`). The first code execution cell will compile (and run) the vector addition program. The second code execution cell will profile the executable that was just compiled using `nsys profile`.

`nsys profile` will generate a `qdrep` report file which can be used in a variety of manners. We use the `--stats=true` flag here to indicate we would like summary statistics printed. There is quite a lot of information printed:

- Profile configuration details
- Report file(s) generation details
- **CUDA API Statistics**
- **CUDA Kernel Statistics**
- **CUDA Memory Operation Statistics (time and size)**
- OS Runtime API Statistics

In this lab you will primarily be using the 3 sections in **bold** above. In the next lab, you will be using the generated report files to give to the Nsight Systems GUI for visual profiling.

After profiling the application, answer the following questions using information displayed in the `CUDA Kernel Statistics` section of the profiling output:

- What was the name of the only CUDA kernel called in this application?
- How many times did this kernel run?
- How long did it take this kernel to run? Record this time somewhere: you will be optimizing this application and will want to know how much faster you can make it.

In [1]:
!mkdir 01-vector-add

In [ ]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
}

int main()
{
  const int N = 2<<20;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  checkCuda( cudaMallocManaged(&a, size) );
  checkCuda( cudaMallocManaged(&b, size) );
  checkCuda( cudaMallocManaged(&c, size) );

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  threadsPerBlock = 256;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;

//  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);
  addVectorsInto<<<1, 1>>>(c, a, b, N);

  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );

  checkElementsAre(7, c, N);

  checkCuda( cudaFree(a) );
  checkCuda( cudaFree(b) );
  checkCuda( cudaFree(c) );
}


Overwriting 01-vector-add/01-vector-add.cu


In [ ]:
!nvcc  -arch=sm_75 -o single-thread-vector-add 01-vector-add/01-vector-add.cu -run

01-vector-add/01-vector-add.cu(64): warning #550-D: variable "numberOfBlocks" was set but never used
    size_t numberOfBlocks;
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [2]:
!apt-get install nsight-systems-2023.3.3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libtinfo5 libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0
  libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0
  libxtst6
The following NEW packages will be installed:
  libtinfo5 libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0
  libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 libxkbcommon-x11-0
  libxtst6 nsight-systems-2023.3.3
0 upgraded, 12 newly installed, 0 to remove and 35 not upgraded.
Need to get 325 MB of archives.
After this operation, 1,269 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libtinfo5 amd64 6.3-2ubuntu0.1 [100 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-xinerama0 amd64 1.14-3ubuntu3 [5,414 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxcb-xinput0 a

In [ ]:
!nsys profile --stats=true ./single-thread-vector-add

Generating '/tmp/nsys-report-bf17.qdstrm'
[1/8] [========================100%] report1.nsys-rep
[2/8] [========================100%] report1.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     91.6      870,969,999         39  22,332,564.1  10,075,989.0     1,952  321,620,576  53,704,903.6  poll                  
      7.1       67,583,143        546     123,778.7      11,382.5       420   19,512,397   1,060,340.4  ioctl                 
      0.8        7,573,163          2   3,786,581.5   3,786,581.5     2,323    7,570,840   5,351,749.7  pthread_cond_broadcast
      0.2        1,792,859         31      57,834.2   

Worth mentioning is that by default, `nsys profile` will not overwrite an existing report file. This is done to prevent accidental loss of work when profiling. If for any reason, you would rather overwrite an existing report file, say during rapid iterations, you can provide the `-f` flag to `nsys profile` to allow overwriting an existing report file.

### Exercise: Optimize and Profile

Take a minute or two to make a simple optimization to [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) by updating its execution configuration so that it runs on many threads in a single thread block. Recompile and then profile with `nsys profile --stats=true` using the code execution cells below. Use the profiling output to check the runtime of the kernel. What was the speed up from this optimization? Be sure to record your results somewhere.

In [ ]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  checkCuda( cudaMallocManaged(&a, size) );
  checkCuda( cudaMallocManaged(&b, size) );
  checkCuda( cudaMallocManaged(&c, size) );

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  threadsPerBlock = 256;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );

  checkElementsAre(7, c, N);

  checkCuda( cudaFree(a) );
  checkCuda( cudaFree(b) );
  checkCuda( cudaFree(c) );
}


Overwriting 01-vector-add/01-vector-add.cu


In [ ]:
!nvcc -arch=sm_75 -o multi-thread-vector-add 01-vector-add/01-vector-add.cu -run

In [ ]:
!rm /tmp/nsys*
!ls -l /tmp/nsys*

rm: cannot remove '/tmp/nsys*': No such file or directory
ls: cannot access '/tmp/nsys*': No such file or directory


In [ ]:
!nsys profile --stats=true ./multi-thread-vector-add

Generating '/tmp/nsys-report-f70c.qdstrm'
[1/8] [========================100%] report3.nsys-rep
[2/8] [========================100%] report3.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report3.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     90.2    1,647,478,770         73  22,568,202.3  10,073,465.0     1,531  318,527,692  44,527,288.5  poll                  
      8.1      147,212,413        546     269,619.8      11,225.0       399  105,834,635   4,546,439.6  ioctl                 
      1.2       21,705,915         24     904,413.1      11,171.5     2,598    8,673,188   2,437,856.5  mmap                  
      0.4        6,951,208          2   3,475,604.0   

### Exercise: Optimize Iteratively

In this exercise you will go through several cycles of editing the execution configuration of [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu), profiling it, and recording the results to see the impact. Use the following guidelines while working:

- Start by listing 3 to 5 different ways you will update the execution configuration, being sure to cover a range of different grid and block size combinations.
- Edit the [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program in one of the ways you listed.
- Compile and profile your updated code with the two code execution cells below.
- Record the runtime of the kernel execution, as given in the profiling output.
- Repeat the edit/profile/record cycle for each possible optimization you listed above

Which of the execution configurations you attempted proved to be the fastest?

In [ ]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
}

int main()
{
  const int N = 2<<20;
  size_t size = N * sizeof(float);

  // Punteros Host (CPU)
  float *h_a, *h_b, *h_c;
  // Punteros Device (GPU)
  float *d_a, *d_b, *d_c;

  // CPU
  h_a = (float*)malloc(size);
  h_b = (float*)malloc(size);
  h_c = (float*)malloc(size);

  // GPU
  checkCuda( cudaMalloc(&d_a, size) );
  checkCuda( cudaMalloc(&d_b, size) );
  checkCuda( cudaMalloc(&d_c, size) );

  initWith(3, h_a, N);
  initWith(4, h_b, N);
  initWith(0, h_c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  threadsPerBlock = 256;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;

  checkCuda( cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice) );
  checkCuda( cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice) );

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(d_c, d_a, d_b, N);

  checkCuda( cudaGetLastError() );

  checkCuda( cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost) );

  checkCuda( cudaDeviceSynchronize() );

  checkElementsAre(7, h_c, N);

  checkCuda( cudaFree(d_a) );
  checkCuda( cudaFree(d_b) );
  checkCuda( cudaFree(d_c) );
}



Overwriting 01-vector-add/01-vector-add.cu


In [ ]:
!nvcc -arch=sm_75 -o iteratively-optimized-vector-add 01-vector-add/01-vector-add.cu -run

In [ ]:
!nsys profile --stats=true ./iteratively-optimized-vector-add

Generating '/tmp/nsys-report-51eb.qdstrm'
[1/8] [========================100%] report5.nsys-rep
[2/8] [========================100%] report5.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report5.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)     Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  -----------  --------  -----------  ------------  ----------------------
     84.5      430,651,093         13  33,127,007.2  2,518,574.0     2,044  330,102,517  90,252,719.1  poll                  
     13.8       70,461,851        543     129,764.0     13,720.0       389   19,837,162   1,045,873.3  ioctl                 
      0.9        4,652,686          2   2,326,343.0  2,326,343.0     3,877    4,648,809   3,284,462.9  pthread_cond_broadcast
      0.4        1,857,789         31      59,928.7     14,

---
## Streaming Multiprocessors and Querying the Device

This section explores how understanding a specific feature of the GPU hardware can promote optimization. After introducing **Streaming Multiprocessors**, you will attempt to further optimize the accelerated vector addition program you have been working on.

The following slides present upcoming material visually, at a high level. Click through the slides before moving on to more detailed coverage of their topics in following sections.

In [ ]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/embedded/task2/NVPROF_UM_1.pptx" width="800px" height="500px" frameborder="0"></iframe></div>

### Streaming Multiprocessors and Warps

The GPUs that CUDA applications run on have processing units called **streaming multiprocessors**, or **SMs**. During kernel execution, blocks of threads are given to SMs to execute. In order to support the GPU's ability to perform as many parallel operations as possible, performance gains can often be had by *choosing a grid size that has a number of blocks that is a multiple of the number of SMs on a given GPU.*

Additionally, SMs create, manage, schedule, and execute groupings of 32 threads from within a block called **warps**. A more [in depth coverage of SMs and warps](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation) is beyond the scope of this course, however, it is important to know that performance gains can also be had by *choosing a block size that has a number of threads that is a multiple of 32.*

### Programmatically Querying GPU Device Properties

In order to support portability, since the number of SMs on a GPU can differ depending on the specific GPU being used, the number of SMs should not be hard-coded into a code bases. Rather, this information should be acquired programatically.

The following shows how, in CUDA C/C++, to obtain a C struct which contains many properties about the currently active GPU device, including its number of SMs:

```cpp
int deviceId;
cudaGetDevice(&deviceId);                  // `deviceId` now points to the id of the currently active GPU.

cudaDeviceProp props;
cudaGetDeviceProperties(&props, deviceId); // `props` now has many useful properties about
                                           // the active GPU device.
```

### Exercise: Query the Device

Currently, [`01-get-device-properties.cu`](../edit/04-device-properties/01-get-device-properties.cu) contains many unassigned variables, and will print gibberish information intended to describe details about the currently active GPU.

Build out [`01-get-device-properties.cu`](../edit/04-device-properties/01-get-device-properties.cu) to print the actual values for the desired device properties indicated in the source code. In order to support your work, and as an introduction to them, use the [CUDA Runtime Docs](http://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html) to help identify the relevant properties in the device props struct. Refer to [the solution](../edit/04-device-properties/solutions/01-get-device-properties-solution.cu) if you get stuck.

In [ ]:
!mkdir  04-device-properties

In [ ]:
%%writefile 04-device-properties/01-get-device-properties.cu

#include <stdio.h>

int main()
{
  /*
   * Assign values to these variables so that the output string below prints the
   * requested properties of the currently active GPU.
   */



int deviceId;
  cudaGetDevice(&deviceId);

  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);

  /*
   * `props` now contains several properties about the current device.
   */

  int computeCapabilityMajor = props.major;
  int computeCapabilityMinor = props.minor;
  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;


  printf("Device ID: %d\nNumber of SMs: %d\nCompute Capability Major: %d\nCompute Capability Minor: %d\nWarp Size: %d\n", deviceId, multiProcessorCount, computeCapabilityMajor, computeCapabilityMinor, warpSize);
}


Writing 04-device-properties/01-get-device-properties.cu


In [ ]:
!nvcc -o get-device-properties 04-device-properties/01-get-device-properties.cu -run

Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32


### Exercise: Optimize Vector Add with Grids Sized to Number of SMs

Utilize your ability to query the device for its number of SMs to refactor the `addVectorsInto` kernel you have been working on inside [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) so that it launches with a grid containing a number of blocks that is a multiple of the number of SMs on the device.

Depending on other specific details in the code you have written, this refactor may or may not improve, or significantly change, the performance of your kernel. Therefore, as always, be sure to use `nsys profile` so that you can quantitatively evaluate performance changes. Record the results with the rest of your findings thus far, based on the profiling output.

In [ ]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i)
  {
    a[i] = num;
  }
}

/*
 * Device kernel stores into `result` the sum of each
 * same-indexed value of `a` and `b`.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in `vector`. This function
 * assumes all values are the same `target` value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  printf("%d \n",N);
  //exit(0);
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  /*
   * nsys should register performance changes when execution configuration
   * is updated.
   */

  //-----------------
  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);

  int computeCapabilityMajor = props.major;
  int computeCapabilityMinor = props.minor;
  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;
  printf("Device ID: %d\nNumber of SMs: %d\nCompute Capability Major: %d\nCompute Capability Minor: %d\nWarp Size: %d\n", deviceId, multiProcessorCount, computeCapabilityMajor, computeCapabilityMinor, warpSize);

  //----------
  threadsPerBlock = 1024;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;

  printf("Minimum Blocks Required: %zu\n", numberOfBlocks);

  printf("threadsPerBlock = %zd    numberOfBlocks =%zd  novo = %zd \n",threadsPerBlock,numberOfBlocks,
  ((numberOfBlocks + props.multiProcessorCount - 1) / props.multiProcessorCount)*props.multiProcessorCount);

  numberOfBlocks = ((numberOfBlocks + props.multiProcessorCount - 1) /  props.multiProcessorCount) * props.multiProcessorCount;

  cudaError_t addVectorsErr;
  cudaError_t asyncErr;

  printf("Threads Per Block: %zu\n", threadsPerBlock);
  printf("Launching with a Grid of %zu Blocks (a multiple of %d SMs)\n", numberOfBlocks, multiProcessorCount);

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  addVectorsErr = cudaGetLastError();
  if(addVectorsErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(addVectorsErr));

  asyncErr = cudaDeviceSynchronize();
  if(asyncErr != cudaSuccess) printf("Error: %s\n", cudaGetErrorString(asyncErr));

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}


Overwriting 01-vector-add/01-vector-add.cu


In [ ]:
!nvcc -arch=sm_75 -o sm-optimized-vector-add 01-vector-add/01-vector-add.cu -run

33554432 
Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32
Minimum Blocks Required: 32768
threadsPerBlock = 1024    numberOfBlocks =32768  novo = 32800 
Threads Per Block: 1024
Launching with a Grid of 32800 Blocks (a multiple of 40 SMs)
Success! All values calculated correctly.


In [ ]:
!nsys profile --stats=true ./sm-optimized-vector-add

33554432 
Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32
Minimum Blocks Required: 32768
threadsPerBlock = 1024    numberOfBlocks =32768  novo = 32800 
Threads Per Block: 1024
Launching with a Grid of 32800 Blocks (a multiple of 40 SMs)
Success! All values calculated correctly.
Generating '/tmp/nsys-report-326a.qdstrm'
[1/8] [========================100%] report1.nsys-rep
[2/8] [========================100%] report1.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)    Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  ---------  -----------  ------------  ----------------------
     89.7    1,663,940,367         73  22,793,703.7  10,073,856.0      1,613  337,746,074  46,328,704

---
## Unified Memory Details

You have been allocating memory intended for use either by host or device code with `cudaMallocManaged` and up until now have enjoyed the benefits of this method - automatic memory migration, ease of programming - without diving into the details of how the **Unified Memory** (**UM**) allocated by `cudaMallocManaged` actual works.

`nsys profile` provides details about UM management in accelerated applications, and using this information, in conjunction with a more-detailed understanding of how UM works, provides additional opportunities to optimize accelerated applications.

The following slides present upcoming material visually, at a high level. Click through the slides before moving on to more detailed coverage of their topics in following sections.

In [ ]:
%%HTML

<div align="center"><iframe src="https://view.officeapps.live.com/op/view.aspx?src=https://developer.download.nvidia.com/training/courses/C-AC-01-V1/embedded/task2/NVPROF_UM_2.pptx" width="800px" height="500px" frameborder="0"></iframe></div>

### Unified Memory Migration

When UM is allocated, the memory is not resident yet on either the host or the device. When either the host or device attempts to access the memory, a [page fault](https://en.wikipedia.org/wiki/Page_fault) will occur, at which point the host or device will migrate the needed data in batches. Similarly, at any point when the CPU, or any GPU in the accelerated system, attempts to access memory not yet resident on it, page faults will occur and trigger its migration.

The ability to page fault and migrate memory on demand is tremendously helpful for ease of development in your accelerated applications. Additionally, when working with data that exhibits sparse access patterns, for example when it is impossible to know which data will be required to be worked on until the application actually runs, and for scenarios when data might be accessed by multiple GPU devices in an accelerated system with multiple GPUs, on-demand memory migration is remarkably beneficial.

There are times - for example when data needs are known prior to runtime, and large contiguous blocks of memory are required - when the overhead of page faulting and migrating data on demand incurs an overhead cost that would be better avoided.

Much of the remainder of this lab will be dedicated to understanding on-demand migration, and how to identify it in the profiler's output. With this knowledge you will be able to reduce the overhead of it in scenarios when it would be beneficial.

### Exercise: Explore UM Migration and Page Faulting

`nsys profile` provides output describing UM behavior for the profiled application. In this exercise, you will make several modifications to a simple application, and make use of `nsys profile` after each change, to explore how UM data migration behaves.

[`01-page-faults.cu`](../edit/06-unified-memory-page-faults/01-page-faults.cu) contains a `hostFunction` and a `gpuKernel`, both which could be used to initialize the elements of a `2<<24` element vector with the number `1`. Currently neither the host function nor GPU kernel are being used.

For each of the 4 questions below, given what you have just learned about UM behavior, first hypothesize about what kind of page faulting should happen, then, edit [`01-page-faults.cu`](../edit/06-unified-memory-page-faults/01-page-faults.cu) to create a scenario, by using one or both of the 2 provided functions in the code bases, that will allow you to test your hypothesis.

In order to test your hypotheses, compile and profile your code using the code execution cells below. Be sure to record your hypotheses, as well as the results, obtained from `nsys profile --stats=true` output. In the output of `nsys profile --stats=true` you should be looking for the following:

- Is there a _CUDA Memory Operation Statistics_ section in the output?
- If so, does it indicate host to device (HtoD) or device to host (DtoH) migrations?
- When there are migrations, what does the output say about how many _Operations_ there were? If you see many small memory migration operations, this is a sign that on-demand page faulting is occurring, with small memory migrations occurring each time there is a page fault in the requested location.

Here are the scenarios for you to explore, along with solutions for them if you get stuck:

- Is there evidence of memory migration and/or page faulting when unified memory is accessed only by the CPU? ([solution](../edit/06-unified-memory-page-faults/solutions/01-page-faults-solution-cpu-only.cu))
- Is there evidence of memory migration and/or page faulting when unified memory is accessed only by the GPU? ([solution](../edit/06-unified-memory-page-faults/solutions/02-page-faults-solution-gpu-only.cu))
- Is there evidence of memory migration and/or page faulting when unified memory is accessed first by the CPU then the GPU? ([solution](../edit/06-unified-memory-page-faults/solutions/03-page-faults-solution-cpu-then-gpu.cu))
- Is there evidence of memory migration and/or page faulting when unified memory is accessed first by the GPU then the CPU? ([solution](../edit/06-unified-memory-page-faults/solutions/04-page-faults-solution-gpu-then-cpu.cu))

In [ ]:
!nvcc -o page-faults 06-unified-memory-page-faults/01-page-faults.cu -run

cc1plus: fatal error: 06-unified-memory-page-faults/01-page-faults.cu: No such file or directory
compilation terminated.


In [ ]:
!nsys profile --stats=true ./page-faults

/bin/bash: line 1: nsys: command not found


### Exercise: Revisit UM Behavior for Vector Add Program

Returning to the [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program you have been working on throughout this lab, review the code bases in its current state, and hypothesize about what kinds of memory migrations and/or page faults you expect to occur. Look at the profiling output for your last refactor (either by scrolling up to find the output or by executing the code execution cell just below), observing the _CUDA Memory Operation Statistics_ section of the profiler output. Can you explain the kinds of migrations and the number of their operations based on the contents of the code base?

In [ ]:
!nsys profile --stats=true ./sm-optimized-vector-add

/bin/bash: line 1: nsys: command not found


### Exercise: Initialize Vector in Kernel

When `nsys profile` gives the amount of time that a kernel takes to execute, the host-to-device page faults and data migrations that occur during this kernel's execution are included in the displayed execution time.

With this in mind, refactor the `initWith` host function in your [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program to instead be a CUDA kernel, initializing the allocated vector in parallel on the GPU. After successfully compiling and running the refactored application, but before profiling it, hypothesize about the following:

- How do you expect the refactor to affect UM memory migration behavior?
- How do you expect the refactor to affect the reported run time of `addVectorsInto`?

Once again, record the results. Refer to [the solution](../edit/07-init-in-kernel/solutions/01-vector-add-init-in-kernel-solution.cu) if you get stuck.

In [6]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

__global__
void initWith(float num, float *a, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    a[i] = num;
  }
}

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  int deviceId;
  cudaGetDevice(&deviceId);
  cudaDeviceProp props;
  cudaGetDeviceProperties(&props, deviceId);

  int computeCapabilityMajor = props.major;
  int computeCapabilityMinor = props.minor;
  int multiProcessorCount = props.multiProcessorCount;
  int warpSize = props.warpSize;
  printf("Device ID: %d\nNumber of SMs: %d\nCompute Capability Major: %d\nCompute Capability Minor: %d\nWarp Size: %d\n", deviceId, multiProcessorCount, computeCapabilityMajor, computeCapabilityMinor, warpSize);

  threadsPerBlock = 1024;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;
  numberOfBlocks = ((numberOfBlocks + props.multiProcessorCount - 1) /  props.multiProcessorCount) * props.multiProcessorCount;

  checkCuda( cudaMallocManaged(&a, size) );
  checkCuda( cudaMallocManaged(&b, size) );
  checkCuda( cudaMallocManaged(&c, size) );

  initWith<<<numberOfBlocks, threadsPerBlock>>>(3, a, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(4, b, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(0, c, N);

  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );

  checkElementsAre(7, c, N);

  checkCuda( cudaFree(a) );
  checkCuda( cudaFree(b) );
  checkCuda( cudaFree(c) );
}


Overwriting 01-vector-add/01-vector-add.cu


In [7]:
!nvcc -arch=sm_75 -o initialize-in-kernel 01-vector-add/01-vector-add.cu -run

Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32


In [8]:
!nsys profile --stats=true ./initialize-in-kernel

Device ID: 0
Number of SMs: 40
Compute Capability Major: 7
Compute Capability Minor: 5
Warp Size: 32
Generating '/tmp/nsys-report-9d58.qdstrm'
[1/8] [========================100%] report2.nsys-rep
[2/8] [========================100%] report2.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report2.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     90.0    1,026,595,483         41  25,038,914.2  10,066,362.0     1,755  459,329,155  72,772,110.1  poll                  
      7.0       79,354,718        557     142,468.1      15,241.0       656   22,062,112   1,159,834.6  ioctl                 
      2.1       23,470,937         24     977,955.7      11,039.5     3,126   10

---
## Asynchronous Memory Prefetching

A powerful technique to reduce the overhead of page faulting and on-demand memory migrations, both in host-to-device and device-to-host memory transfers, is called **asynchronous memory prefetching**. Using this technique allows programmers to asynchronously migrate unified memory (UM) to any CPU or GPU device in the system, in the background, prior to its use by application code. By doing this, GPU kernels and CPU function performance can be increased on account of reduced page fault and on-demand data migration overhead.

Prefetching also tends to migrate data in larger chunks, and therefore fewer trips, than on-demand migration. This makes it an excellent fit when data access needs are known before runtime, and when data access patterns are not sparse.

CUDA Makes asynchronously prefetching managed memory to either a GPU device or the CPU easy with its `cudaMemPrefetchAsync` function. Here is an example of using it to both prefetch data to the currently active GPU device, and then, to the CPU:

```cpp
int deviceId;
cudaGetDevice(&deviceId);                                         // The ID of the currently active GPU device.

cudaMemPrefetchAsync(pointerToSomeUMData, size, deviceId);        // Prefetch to GPU device.
cudaMemPrefetchAsync(pointerToSomeUMData, size, cudaCpuDeviceId); // Prefetch to host. `cudaCpuDeviceId` is a
                                                                  // built-in CUDA variable.
```

### Exercise: Prefetch Memory

At this point in the lab, your [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) program should not only be launching a CUDA kernel to add 2 vectors into a third solution vector, all which are allocated with `cudaMallocManaged`, but should also be initializing each of the 3 vectors in parallel in a CUDA kernel. If for some reason, your application does not do any of the above, please refer to the following [reference application](../edit/07-init-in-kernel/solutions/01-vector-add-init-in-kernel-solution.cu), and update your own code bases to reflect its current functionality.

Conduct 3 experiments using `cudaMemPrefetchAsync` inside of your [01-vector-add.cu](../edit/01-vector-add/01-vector-add.cu) application to understand its impact on page-faulting and memory migration.

- What happens when you prefetch one of the initialized vectors to the device?
- What happens when you prefetch two of the initialized vectors to the device?
- What happens when you prefetch all three of the initialized vectors to the device?

Hypothesize about UM behavior, page faulting specifically, as well as the impact on the reported run time of the initialization kernel, before each experiment, and then verify by running `nsys profile`. Refer to [the solution](../edit/08-prefetch/solutions/01-vector-add-prefetch-solution.cu) if you get stuck.

In [6]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

__global__
void initWith(float num, float *a, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    a[i] = num;
  }
}

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  int deviceId;
  checkCuda( cudaGetDevice(&deviceId) );
  cudaDeviceProp props;
  checkCuda( cudaGetDeviceProperties(&props, deviceId) );

  printf("Device ID: %d\nNumber of SMs: %d\n", deviceId, props.multiProcessorCount);

  threadsPerBlock = 1024;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;
  numberOfBlocks = ((numberOfBlocks + props.multiProcessorCount - 1) /  props.multiProcessorCount) * props.multiProcessorCount;

  checkCuda( cudaMallocManaged(&a, size) );
  checkCuda( cudaMallocManaged(&b, size) );
  checkCuda( cudaMallocManaged(&c, size) );

  checkCuda( cudaMemPrefetchAsync(a, size, deviceId, 0) );
  checkCuda( cudaMemPrefetchAsync(b, size, deviceId, 0) );
  checkCuda( cudaMemPrefetchAsync(c, size, deviceId, 0) );

  initWith<<<numberOfBlocks, threadsPerBlock>>>(3, a, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(4, b, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(0, c, N);

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  //checkCuda( cudaMemPrefetchAsync(c, size, cudaCpuDeviceId, 0) );

  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );

  checkElementsAre(7, c, N);

  checkCuda( cudaFree(a) );
  checkCuda( cudaFree(b) );
  checkCuda( cudaFree(c) );
}


Overwriting 01-vector-add/01-vector-add.cu


In [7]:
!nvcc -arch=sm_75 -o prefetch-to-gpu 01-vector-add/01-vector-add.cu -run

Device ID: 0
Number of SMs: 40


In [8]:
!nsys profile --stats=true ./prefetch-to-gpu

Device ID: 0
Number of SMs: 40
Generating '/tmp/nsys-report-8ce7.qdstrm'
[1/8] [========================100%] report2.nsys-rep
[2/8] [========================100%] report2.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report2.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     88.0      698,378,820         32  21,824,338.1  10,076,266.5     2,513  334,772,863  59,428,660.0  poll                  
      8.1       64,226,683        560     114,690.5      12,706.5       544   19,246,166     895,370.3  ioctl                 
      2.2       17,204,124         24     716,838.5      11,434.0     2,398   15,318,077   3,117,381.6  mmap                  
      1.1        8,487,

### Exercise: Prefetch Memory Back to the CPU

Add additional prefetching back to the CPU for the function that verifies the correctness of the `addVectorInto` kernel. Again, hypothesize about the impact on UM before profiling in `nsys` to confirm. Refer to [the solution](../edit/08-prefetch/solutions/02-vector-add-prefetch-solution-cpu-also.cu) if you get stuck.

In [9]:
%%writefile 01-vector-add/01-vector-add.cu

#include <stdio.h>
#include <assert.h>

inline cudaError_t checkCuda(cudaError_t result)
{
  if (result != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime Error: %s\n", cudaGetErrorString(result));
    assert(result == cudaSuccess);
  }
  return result;
}

__global__
void initWith(float num, float *a, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    a[i] = num;
  }
}

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride)
  {
    result[i] = a[i] + b[i];
  }
}

void checkElementsAre(float target, float *array, int N)
{
  for(int i = 0; i < N; i++)
  {
    if(array[i] != target)
    {
      printf("FAIL: array[%d] - %0.0f does not equal %0.0f\n", i, array[i], target);
      exit(1);
    }
  }
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a;
  float *b;
  float *c;

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  int deviceId;
  checkCuda( cudaGetDevice(&deviceId) );
  cudaDeviceProp props;
  checkCuda( cudaGetDeviceProperties(&props, deviceId) );

  printf("Device ID: %d\nNumber of SMs: %d\n", deviceId, props.multiProcessorCount);

  threadsPerBlock = 1024;
  numberOfBlocks = (N + threadsPerBlock - 1) / threadsPerBlock;
  numberOfBlocks = ((numberOfBlocks + props.multiProcessorCount - 1) /  props.multiProcessorCount) * props.multiProcessorCount;

  checkCuda( cudaMallocManaged(&a, size) );
  checkCuda( cudaMallocManaged(&b, size) );
  checkCuda( cudaMallocManaged(&c, size) );

  checkCuda( cudaMemPrefetchAsync(a, size, deviceId, 0) );
  checkCuda( cudaMemPrefetchAsync(b, size, deviceId, 0) );
  checkCuda( cudaMemPrefetchAsync(c, size, deviceId, 0) );

  initWith<<<numberOfBlocks, threadsPerBlock>>>(3, a, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(4, b, N);
  initWith<<<numberOfBlocks, threadsPerBlock>>>(0, c, N);

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  checkCuda( cudaMemPrefetchAsync(c, size, cudaCpuDeviceId, 0) );

  checkCuda( cudaGetLastError() );
  checkCuda( cudaDeviceSynchronize() );

  checkElementsAre(7, c, N);

  checkCuda( cudaFree(a) );
  checkCuda( cudaFree(b) );
  checkCuda( cudaFree(c) );
}


Overwriting 01-vector-add/01-vector-add.cu


In [10]:
!nvcc -arch=sm_75 -o prefetch-to-cpu 01-vector-add/01-vector-add.cu -run

Device ID: 0
Number of SMs: 40


In [11]:
!nsys profile --stats=true ./prefetch-to-cpu

Device ID: 0
Number of SMs: 40
Generating '/tmp/nsys-report-c777.qdstrm'
[1/8] [========================100%] report3.nsys-rep
[2/8] [========================100%] report3.sqlite
[3/8] Executing 'nvtx_sum' stats report
SKIPPED: /content/report3.sqlite does not contain NV Tools Extension (NVTX) data.
[4/8] Executing 'osrt_sum' stats report

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)    Min (ns)   Max (ns)    StdDev (ns)            Name         
 --------  ---------------  ---------  ------------  ------------  --------  -----------  ------------  ----------------------
     85.2      649,636,799         27  24,060,622.2  10,072,145.0     2,585  328,753,129  62,904,914.2  poll                  
     10.9       82,805,625        561     147,603.6      13,250.0       574   18,302,230   1,102,760.0  ioctl                 
      1.0        7,613,750          3   2,537,916.7       2,646.0     2,505    7,608,599   4,391,339.7  pthread_cond_broadcast
      0.9        6,533,

After this series of refactors to use asynchronous prefetching, you should see that there are fewer, but larger, memory transfers, and, that the kernel execution time is significantly decreased.

---
## Summary

At this point in the lab, you are able to:

- Use the Nsight Systems command line tool (**nsys**) to profile accelerated application performance.
- Leverage an understanding of **Streaming Multiprocessors** to optimize execution configurations.
- Understand the behavior of **Unified Memory** with regard to page faulting and data migrations.
- Use **asynchronous memory prefetching** to reduce page faults and data migrations for increased performance.
- Employ an iterative development cycle to rapidly accelerate and deploy applications.

In order to consolidate your learning, and reinforce your ability to iteratively accelerate, optimize, and deploy applications, please proceed to this lab's final exercise. After completing it, for those of you with time and interest, please proceed to the *Advanced Content* section.

---
## Final Exercise: Iteratively Optimize an Accelerated SAXPY Application

A basic accelerated SAXPY (Single Precision a\*x+b) application has been provided for you [here](../edit/09-saxpy/01-saxpy.cu). It currently contains a couple of bugs that you will need to find and fix before you can successfully compile, run, and then profile it with `nsys profile`.

After fixing the bugs and profiling the application, record the runtime of the `saxpy` kernel and then work *iteratively* to optimize the application, using `nsys profile` after each iteration to notice the effects of the code changes on kernel performance and UM behavior.

Utilize the techniques from this lab. To support your learning, utilize [effortful retrieval](http://sites.gsu.edu/scholarlyteaching/effortful-retrieval/) whenever possible, rather than rushing to look up the specifics of techniques from earlier in the lesson.

Your end goal is to profile an accurate `saxpy` kernel, without modifying `N`, to run in under *200us*. Check out [the solution](../edit/09-saxpy/solutions/02-saxpy-solution.cu) if you get stuck, and feel free to compile and profile it if you wish.

In [ ]:
!nvcc -o saxpy 09-saxpy/01-saxpy.cu -run

cc1plus: fatal error: 09-saxpy/01-saxpy.cu: No such file or directory
compilation terminated.


In [ ]:
!nsys profile --stats=true ./saxpy

/bin/bash: line 1: nsys: command not found
